In [1]:
#Biblioteka Efficient-Apriori służaca do obliczania reguł asocjacyjnych algorytmem Apriori 

#Instalacja:
#pip install efficient-apriori

#Dokumentacja:
#https://efficient-apriori.readthedocs.io/en/latest/

#Oto minimalny przykład roboczy.

#Polecenie usuwa wszystkie zmienne z pamięci 
%reset -f 

from efficient_apriori import apriori

#Wejściowa liosta krotek
transactions = [('jajko', 'boczek', 'zupa'),
                ('jajko', 'boczek', 'jabłko'),
                ('zupa', 'boczek', 'banan')]

itemsets, rules = apriori(transactions, min_support=0.5, min_confidence=1)

print(rules) 

[{jajko} -> {boczek}, {zupa} -> {boczek}]


In [5]:
#Jeśli dane do liczenia reguł są w pliku csv w innym formacie, to należy je przekonwertować na listę krotek, gdzie każda krotka 
#przechowuje jeden koszyk zakupowy
#Ten przykład dotyczy formatu, gdy w wierszach podajemy produkty, ale także numery transakcji (koszyka) i ewentualnie inne kody
#Np:
# Transakcja,Produkt,Kod
# 1,brokuly,3
# 1,zielona papryka,5
# 1,kukurydza,4
# 2,szparagi,1
# 2,kabaczki,6

#Polecenie usuwa wszystkie zmienne z pamięci 
%reset -f 

import pandas as pd
from efficient_apriori import apriori

#Konwersja tablicy danych na listę krotek

def getTupleList(dataset,transaction_attr_index,product_attr_index):
   
    noRow = dataset.shape[0]
    
    output_tuple_list = []
    
    current_transaction_id = None
    current_product_list = []
    for i in range(0, noRow): #Przegladanie danych wejsciowych
        
        transaction_id = dataset.iat[i, transaction_attr_index]
        product = str(dataset.iat[i, product_attr_index])
        
        if current_transaction_id is None or current_transaction_id != transaction_id:
            
            if len(current_product_list)>0:
                locTuple = tuple(current_product_list)
                output_tuple_list.append(locTuple)                   
            
            current_transaction_id = transaction_id
            current_product_list = []
            current_product_list.append(product)
        else:
            current_product_list.append(product)
      
    if len(current_product_list)>0:
        locTuple = tuple(current_product_list)
        output_tuple_list.append(locTuple)   
                    
            
    return output_tuple_list      



dataset = pd.read_csv('./dane/warzywa.csv',sep=',') #Odczytanie zbioru danych

print(dataset.head(3))
print("---------------------------------")

transactions = getTupleList(dataset,0,1)

#Obliczenie reguł asocjacyjnych        
itemsets, rules = apriori(transactions, min_support=0.2, min_confidence=0.6)

#Wypisanie reguł
for i in range(0,len(rules)):
    rule = rules[i]
    left_side = list(rule.lhs)
    right_side = list(rule.rhs)
    confidence = rule.confidence
    support = rule.support
    print(i+1,left_side,"=>",right_side,confidence,support) #Wypisanie reguły



   Transakcja          Produkt  Kod
0           1          brokuly    3
1           1  zielona papryka    5
2           1        kukurydza    4
---------------------------------
1 ['brokuly'] => ['fasola'] 0.6 0.21428571428571427
2 ['zielona papryka'] => ['brokuly'] 0.8 0.2857142857142857
3 ['brokuly'] => ['zielona papryka'] 0.8 0.2857142857142857
4 ['kabaczki'] => ['fasola'] 0.8571428571428571 0.42857142857142855
5 ['fasola'] => ['kabaczki'] 0.6 0.42857142857142855
6 ['kukurydza'] => ['fasola'] 0.625 0.35714285714285715
7 ['pomidory'] => ['fasola'] 0.6666666666666666 0.2857142857142857
8 ['szparagi'] => ['fasola'] 0.8333333333333334 0.35714285714285715
9 ['zielona papryka'] => ['fasola'] 0.6 0.21428571428571427
10 ['szparagi'] => ['kabaczki'] 0.8333333333333334 0.35714285714285715
11 ['kabaczki'] => ['szparagi'] 0.7142857142857143 0.35714285714285715
12 ['pomidory'] => ['kukurydza'] 0.6666666666666666 0.2857142857142857
13 ['zielona papryka'] => ['kukurydza'] 0.6 0.21428571428571427
1

In [4]:
#Możliwe jest także filtrowanie i sortowanie zwróconej listy reguł asocjacyjnych.

#Polecenie usuwa wszystkie zmienne z pamięci 
%reset -f 


import pandas as pd
from efficient_apriori import apriori

#Konwersja tablicy danych na listę krotek

def getTupleList(dataset,transaction_attr_index,product_attr_index):
   
    noRow = dataset.shape[0]
    
    output_tuple_list = []
    
    current_transaction_id = None
    current_product_list = []
    for i in range(0, noRow):
        
        transaction_id = dataset.iat[i, transaction_attr_index]
        product = str(dataset.iat[i, product_attr_index])
        
        if current_transaction_id is None or current_transaction_id != transaction_id:
            
            if len(current_product_list)>0:
                locTuple = tuple(current_product_list)
                output_tuple_list.append(locTuple)                   
            
            current_transaction_id = transaction_id
            current_product_list = []
            current_product_list.append(product)
        else:
            current_product_list.append(product)
      
    if len(current_product_list)>0:
        locTuple = tuple(current_product_list)
        output_tuple_list.append(locTuple)   
                    
            
    return output_tuple_list      



dataset = pd.read_csv('./dane/warzywa.csv',sep=',') #Odczytanie zbioru danych

#print(dataset.head(3))

transactions = getTupleList(dataset,0,1)

#Obliczenie reguł asocjacyjnych        
itemsets, rules = apriori(transactions, min_support=0.2, min_confidence=0.6)
   
#Wybierz reguły z 2 deskryptorami po lewej stronie i jednym po prawej stronie
#Posortuj według confidence malejąco
#Warunek do selekcji oraz warunek do sortowania definioujemy za pomocą wyrażeń lambda

selected_rules = filter(lambda rule: len(rule.lhs) == 2 and len(rule.rhs) == 1, rules)

for rule in sorted(selected_rules, key=lambda rule: rule.confidence, reverse=True):
  print(rule)  
    
print("KONIEC")    
    

{kabaczki, szparagi} -> {fasola} (conf: 0.800, supp: 0.286, lift: 1.120, conv: 1.429)
{fasola, szparagi} -> {kabaczki} (conf: 0.800, supp: 0.286, lift: 1.600, conv: 2.500)
{kukurydza, pomidory} -> {fasola} (conf: 0.750, supp: 0.214, lift: 1.050, conv: 1.143)
{fasola, pomidory} -> {kukurydza} (conf: 0.750, supp: 0.214, lift: 1.312, conv: 1.714)
{fasola, kabaczki} -> {szparagi} (conf: 0.667, supp: 0.286, lift: 1.556, conv: 1.714)
{fasola, kukurydza} -> {pomidory} (conf: 0.600, supp: 0.214, lift: 1.400, conv: 1.429)
KONIEC


In [2]:
#Jeśli dane do liczenia reguł są w pliku csv w innym formacie, to należy je przekonwertować na listę krotek, gdzie każda krotka 
#przechowuje jeden koszyk zakupowy
#Ten przykład dotyczy formatu, gdy w wierszach podajemy koszyki poprzez wyszczególnienie czy w koszyku jest dany produkt, 
#czy go niema

#Np:
# chleb, mleko, cukier, maslo, platki, jajka
# nie, nie, tak, nie, tak, nie
# nie, tak, nie, tak, tak, tak
# nie, tak, tak, nie, tak, tak

#Polecenie usuwa wszystkie zmienne z pamięci 
%reset -f 

import pandas as pd
from efficient_apriori import apriori

#Konwersja tablicy danych na listę krotek

def getTupleList2(dataset):
   
    noRow = dataset.shape[0]
    noColumn = dataset.shape[1]    
        
    column_list = list(dataset.columns.values) #Pobranie listy nazw kolumn
            
    output_tuple_list = []
        
    for i in range(0, noRow): #Przegladanie wierszy                
        basket = []        
        for j in range(0, noColumn): #Przegladanie kolumn
            value = dataset.iat[i, j]               
            if value=="tak":                
                basket.append(column_list[j])
        
        if len(basket)>0:
            locTuple = tuple(basket)
            output_tuple_list.append(locTuple)       
            
    return output_tuple_list      



dataset = pd.read_csv('./dane/koszyki.csv',sep=',') #Odczytanie zbioru danych

print(dataset.head(3))
print("---------------------------------")

transactions = getTupleList2(dataset)

print("---------------------------------")

#Obliczenie reguł asocjacyjnych        

minimalne_wsparcie = 0.2 #Poprzedniki wyznaczonych reguł mają pasować do co najmniej 20 procent wszystkich koszyków (wierszy)
minimalna_ufnosc = 0.8  #Co najmniej 80 procent koszyków pasujących do poprzednika, pasuje również do nastepnika

itemsets, rules = apriori(transactions, min_support=minimalne_wsparcie, min_confidence=minimalna_ufnosc)

#Wypisanie reguł
for i in range(0,len(rules)):
    rule = rules[i]
    left_side = list(rule.lhs)
    right_side = list(rule.rhs)
    confidence = rule.confidence
    support = rule.support
    print(i+1,left_side,"=>",right_side,confidence,round(support,3)) #Wypisanie reguły



  chleb mleko cukier maslo platki jajka
0   nie   nie    tak   nie    tak   nie
1   nie   tak    nie   tak    tak   tak
2   nie   tak    tak   nie    tak   tak
---------------------------------
---------------------------------
1 ['chleb'] => ['cukier'] 0.8 0.267
2 ['chleb'] => ['jajka'] 0.8 0.267
3 ['chleb'] => ['maslo'] 0.8 0.267
4 ['platki'] => ['cukier'] 0.8181818181818182 0.6
5 ['mleko'] => ['platki'] 1.0 0.467
6 ['chleb', 'maslo'] => ['jajka'] 1.0 0.267
7 ['chleb', 'jajka'] => ['maslo'] 1.0 0.267
8 ['chleb'] => ['jajka', 'maslo'] 0.8 0.267
9 ['cukier', 'mleko'] => ['platki'] 1.0 0.333
10 ['jajka', 'mleko'] => ['platki'] 1.0 0.333
11 ['maslo', 'mleko'] => ['platki'] 1.0 0.267
12 ['chleb', 'cukier', 'maslo'] => ['jajka'] 1.0 0.2
13 ['chleb', 'cukier', 'jajka'] => ['maslo'] 1.0 0.2
14 ['cukier', 'jajka', 'mleko'] => ['platki'] 1.0 0.2
15 ['jajka', 'maslo', 'mleko'] => ['platki'] 1.0 0.2


In [124]:
#Kierownik sklepu spożywczego zdecydował się dokonać sprawdzenia rozlokowania towarów na terenie sklepu w ten sposób, 
#aby zwiększyć sprzedaż. W tym celu postanowił zlecić wykonanie eksploracji swoich danych. Ponieważ dane, którymi 
#dysponuje dotyczą transakcji dokonywanych na kasach fiskalnych (zawartość paragonów), naturalną metodą eksploracji 
#pasująca do tych danych jest generowanie reguł asocjacyjnych pomiędzy produktami w koszyku zakupów. 
#W związku z tym zostałeś poproszony o przeprowadzenie eksploracji tą metodą oraz o wykonanie kilku poniższych poleceń. 
#Reguły asocjacyjne mają być liczone metodą Apriori, przy czym mają być wyliczone wszystkie reguły o wsparciu co najmniej
#20 procent wierszy i ufności co najmniej 0.6. 

#a) podać regułę (lub reguły) o ufności 1.0 i wsparciu co najmniej 0.3 oraz średnią ufność wszystkich wyliczonych 
#reguł asocjacyjnych i ich średnie wsparcie.

#b) wykorzystując wyliczone reguły asocjacyjne, podać produkty, które powinny być blisko produktu „płatki” zakładając, 
#że są to produkty, występujące w następnikach reguł o wsparciu nie mniejszym niż 0.7, ufności co najmniej 0.3 i mających 
#w poprzedniku produkt „płatki”,

#c) wykorzystując wyliczone reguły asocjacyjne podać produkty, które silnie wspierają sprzedaż produktu jajka, 
#tzn. jajka powinny być umieszczone blisko tych produktów (brać pod uwagę reguły ze wsparciem co najmniej 0.3 i ufności 
#co najmniej 0.6 z produktem "jajka" w następniku).

import efficient_apriori
import pandas as pd

data = pd.read_csv('koszyki.csv', sep=',')

kolumny = ['chleb', 'mleko', 'cukier', 'maslo', 'platki', 'jajka']

for c in kolumny:
    data[c] = data[c].map({"tak": c, "nie": " "})

krotki_pofiltrowane = []

for _, row in data.iterrows():
    produkty = [p for p in row[kolumny].tolist() if p.strip() != ""]
    krotki_pofiltrowane.append(tuple(produkty))

items, rules = efficient_apriori.apriori(krotki_pofiltrowane,min_support=0.2, min_confidence=0.6)

#a
rules_a = [r for r in rules if r.confidence == 1.0 and r.support >= 0.3]

rules_avg_conf = 0
rules_avg_support = 0

for i in rules:
    rules_avg_conf += i.confidence
    rules_avg_support += i.support

rules_avg_conf /= len(rules)
rules_avg_support /= len(rules)

print(f'Regóły o ufności 1.0 i minimalnym wsparciu przynajmniej 0.3: {rules_a}')
print(f"Średnia wartość ufności dla WSZYSTKICH regół: {rules_avg_conf}")
print(f"Średnia wartość wsparcia dla WSZYSTKICH regół: {rules_avg_support}")

#b
produkty_blisko_platki = []

for r in rules:
    if 'platki' in r.lhs and r.support >= 0.7 and r.confidence >= 0.3:
        produkty_blisko_platki.extend(list(r.lhs))

print(f"Produkty które powinny stać blisko płatków: {produkty_blisko_platki}")

#c
produkty_jajka = []

for r in rules:
    if 'jajka' in r.rhs and r.confidence >= 0.6:
        if r.support >= 0.3:
            produkty_jajka.extend(list(r.lhs))

produkty_jajka = list(set(produkty_jajka))

print(f"Produkty które powinny stać blisko jajek: {produkty_jajka}")


Regóły o ufności 1.0 i minimalnym wsparciu przynajmniej 0.3: [{mleko} -> {platki}, {cukier, mleko} -> {platki}, {jajka, mleko} -> {platki}]
Średnia wartość ufności dla WSZYSTKICH regół: 0.7282643032643038
Średnia wartość wsparcia dla WSZYSTKICH regół: 0.2927927927927926
Produkty które powinny stać blisko płatków: []
Produkty które powinny stać blisko jajek: ['platki', 'maslo', 'cukier', 'mleko']
